In [89]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

from pyspark.sql import SparkSession

In [90]:
spark = SparkSession.builder.appName("DataLoadTest").getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("INFO")

In [91]:
sys.executable

'D:\\Users\\pereg\\AppData\\Local\\Programs\\Python\\Python311\\python.exe'

In [92]:
path = "../../data/dom/amazon.dataset.libsvm.11.24.50.txt"
lines = sc.textFile(path)

In [93]:
c = lines.count()

In [94]:
c

5374

In [95]:
metadata = lines.filter(lambda line: line[0] == "#")

In [96]:
metadata

PythonRDD[3] at RDD at PythonRDD.scala:53

In [97]:
rdd = lines\
    .filter(lambda line: line[0] != "#")\
    .map(lambda line: (line.split(" ^|^ ")))\
    .filter(lambda record: len(record) == 4)

In [98]:
rdd.top(5)

[['-1',
  ' 1:999.7 2:1501.1 3:42.8 4:15.3 5:6 11:31 12:3474 33:6 34:6 35:6 64:42.8 65:15.3 70:999.7 71:1501.1 72:42.8 73:15.3 74:6 75:1 78:2 80:30 81:3473 82:253.9683 90:999.7 91:1501.1 95:1 102:6 103:6 104:6 105:1 109:3473 122:42.8 124:15.3 133:42.8 134:15.3 139:997.7 140:1364.3 141:179.6 142:20 143:5 149:30 150:3472 171:5 172:5 173:5 202:179.6 203:20 208:997.7 209:1364.3 210:179.6 211:20 212:5 218:30 219:3472 240:5 241:5 242:5 271:179.6 272:20',
  '$17.99',
  'https://www.amazon.com/dp/B0148NNKTC'],
 ['-1',
  ' 1:999.7 2:1369.3 3:107.7 4:15.3 5:17 11:31 12:3471 33:17 34:17 35:17 64:107.7 65:15.3 70:999.7 71:1369.3 72:107.7 73:15.3 74:17 75:1 77:1 78:2 80:30 81:3470 82:99.6885 90:999.7 91:1369.3 95:1 102:17 103:17 104:17 105:1 109:3470 122:107.7 124:15.3 126:107.7 128:15.3 133:107.7 134:15.3 135:107.7 136:15.3 139:997.7 140:1364.3 141:179.6 142:20 149:30 150:3469 208:997.7 209:1364.3 210:179.6 211:20 218:30 219:3469',
  '4-Year Protection',
  'https://www.amazon.com/dp/B0148NNKTC'],


In [99]:
rdd2 = rdd.map(lambda r: (r[0], r[1], r[2].split(" "), r[3]))
df2 = rdd2.toDF(["label", "numeric_features", "words", "url"])

In [100]:
df2.show(5)

+-----+--------------------+--------------------+--------------------+
|label|    numeric_features|               words|                 url|
+-----+--------------------+--------------------+--------------------+
|   -1| 1:231 2:1348.3 3...|[Prime, Exclusive...|https://www.amazo...|
|   -1| 1:261.3 2:1348.3...|            [$21.59]|https://www.amazo...|
|   -1| 1:258.6 2:1348.3...|                 [$]|https://www.amazo...|
|   -1| 1:253.7 2:1356 3...|                [21]|https://www.amazo...|
|   -1| 1:253.7 2:1388.8...|                 [.]|https://www.amazo...|
+-----+--------------------+--------------------+--------------------+
only showing top 5 rows



In [101]:
from pyspark.ml.feature import Word2Vec
from ml.dom.data.DataUtils import DataUtils
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="words", outputCol="word_vec")
model = word2Vec.fit(df2)
df3 = model.transform(df2)

In [102]:
df3

DataFrame[label: string, numeric_features: string, words: array<string>, url: string, word_vec: vector]

In [103]:
df3.show(5)

+-----+--------------------+--------------------+--------------------+--------------------+
|label|    numeric_features|               words|                 url|            word_vec|
+-----+--------------------+--------------------+--------------------+--------------------+
|   -1| 1:231 2:1348.3 3...|[Prime, Exclusive...|https://www.amazo...|[0.16822208464145...|
|   -1| 1:261.3 2:1348.3...|            [$21.59]|https://www.amazo...|[-0.1416766792535...|
|   -1| 1:258.6 2:1348.3...|                 [$]|https://www.amazo...|[-0.0205316543579...|
|   -1| 1:253.7 2:1356 3...|                [21]|https://www.amazo...|[-0.0653516799211...|
|   -1| 1:253.7 2:1388.8...|                 [.]|https://www.amazo...|[-0.4197035133838...|
+-----+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [104]:
spark.stop()